# Films Data Analysis Report
## Prodivi Style Analytics Dashboard

This notebook performs comprehensive film data analysis using Polars library, implementing a Star Schema data model with interactive visualizations.

### Setup and Imports

In [1]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Prodivi color palette
PRODIVI_COLORS = {
    'primary': '#1E3A5F',      # Dark blue
    'secondary': '#3498DB',    # Light blue
    'accent': '#E74C3C',       # Red for highlights
    'success': '#27AE60',      # Green
    'warning': '#F39C12',      # Orange
    'neutral': '#95A5A6',      # Gray
    'background': '#F8F9FA',   # Light background
    'text': '#2C3E50'          # Dark text
}

# File paths
FILMS_PATH = r"D:\Downloads\Films.xlsx"
GREATEST_FILMS_PATH = r"D:\Downloads\TSPDT - 1,000 Greatest Films (Table) (1).xlsx"

print("Setup complete!")

Setup complete!


---
## Task 1: Load Films.xlsx Tables
Loading the main Films table along with Certificate and Genre dimension tables.

In [2]:
# Load all sheets from Films.xlsx
df_films = pl.read_excel(FILMS_PATH, sheet_name='Films')
df_certificate = pl.read_excel(FILMS_PATH, sheet_name='Certificate')
df_genre = pl.read_excel(FILMS_PATH, sheet_name='Genre')

# Fix data types - convert ID columns to Int64 to match Films table
df_certificate = df_certificate.with_columns(
    pl.col('CertificateID').cast(pl.Int64)
)
df_genre = df_genre.with_columns(
    pl.col('GenreID').cast(pl.Int64)
)

print("=" * 60)
print("FILMS TABLE")
print("=" * 60)
print(f"Shape: {df_films.shape}")
print(f"Columns: {df_films.columns}")
print("\nSample data:")
df_films.head(5)

FILMS TABLE
Shape: (1000, 15)
Columns: ['FilmID', 'Title', 'ReleaseDate', 'DirectorID', 'StudioID', 'Review', 'CountryID', 'LanguageID', 'GenreID', 'RunTimeMinutes', 'CertificateID', 'BudgetDollars', 'BoxOfficeDollars', 'OscarNominations', 'OscarWins']

Sample data:


FilmID,Title,ReleaseDate,DirectorID,StudioID,Review,CountryID,LanguageID,GenreID,RunTimeMinutes,CertificateID,BudgetDollars,BoxOfficeDollars,OscarNominations,OscarWins
i64,str,date,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Jurassic Park""",1993-06-11,4,41,"""People clone dinosaurs. Dinosa…",241,1,8,126,2,63000000,1029939903,3,3
2,"""Spider-Man""",2002-05-03,11,53,"""High-school nerd Peter Parker …",241,1,4,121,4,140000000,821708551,2,0
3,"""King Kong""",2005-12-14,12,42,"""Dizzy blonde actress falls for…",167,1,8,187,4,207000000,550500000,4,3
5,"""Superman Returns""",2006-07-14,14,43,"""The Man of Steel returns from …",241,1,4,154,4,204000000,391081192,1,0
6,"""Titanic""",1998-01-23,15,4,"""Bleurgh! Sorry, that's the sou…",241,1,6,194,3,200000000,2186772302,14,11


In [3]:
print("=" * 60)
print("CERTIFICATE TABLE (Dimension)")
print("=" * 60)
print(f"Shape: {df_certificate.shape}")
df_certificate

CERTIFICATE TABLE (Dimension)
Shape: (7, 2)


CertificateID,Certificate
i64,str
1,"""U"""
2,"""PG"""
3,"""12"""
4,"""12A"""
5,"""15"""
6,"""18"""
7,"""Unknown"""


In [4]:
print("=" * 60)
print("GENRE TABLE (Dimension)")
print("=" * 60)
print(f"Shape: {df_genre.shape}")
df_genre

GENRE TABLE (Dimension)
Shape: (24, 2)


GenreID,Genre
i64,str
1,"""Western"""
2,"""Drama"""
3,"""Science Fiction"""
4,"""Action"""
5,"""War"""
…,…
24,"""Documentary"""
25,"""Disaster"""
26,"""Family"""


---
## Task 2: Load and Clean Greatest Films Table
Loading the TSPDT 1,000 Greatest Films table and cleaning errors.

In [5]:
# Load Greatest Films raw data
df_greatest_raw = pl.read_excel(GREATEST_FILMS_PATH)
print("Raw data shape:", df_greatest_raw.shape)
print("Raw columns:", df_greatest_raw.columns)
df_greatest_raw.head(5)

Raw data shape: (1001, 7)
Raw columns: ['TSPDT - 1,000 Greatest Films (Table)', '__UNNAMED__1', '__UNNAMED__2', '__UNNAMED__3', '__UNNAMED__4', '__UNNAMED__5', '__UNNAMED__6']


"TSPDT - 1,000 Greatest Films (Table)",__UNNAMED__1,__UNNAMED__2,__UNNAMED__3,__UNNAMED__4,__UNNAMED__5,__UNNAMED__6
str,i64,str,str,str,str,str
"""Pos""",2025,"""Title""","""Director""","""Year""","""Country""","""Mins"""
"""1""",1,"""Citizen Kane""","""Welles, Orson""","""1941""","""USA""","""119"""
"""2""",2,"""Vertigo""","""Hitchcock, Alfred""","""1958""","""USA""","""128"""
"""3""",3,"""2001: A Space Odyssey""","""Kubrick, Stanley""","""1968""","""UK""","""139"""
"""4""",4,"""Tokyo Story""","""Ozu, Yasujiro""","""1953""","""Japan""","""134"""


In [6]:
# Clean the Greatest Films table
# The first row contains actual column names
df_greatest_films = df_greatest_raw.slice(1)  # Skip header row

# Rename columns based on first row content
df_greatest_films = df_greatest_films.rename({
    'TSPDT - 1,000 Greatest Films (Table)': 'Position',
    '__UNNAMED__1': 'Rank2025',
    '__UNNAMED__2': 'Title',
    '__UNNAMED__3': 'Director',
    '__UNNAMED__4': 'Year',
    '__UNNAMED__5': 'Country',
    '__UNNAMED__6': 'Minutes'
})

# Convert data types and clean
df_greatest_films = df_greatest_films.with_columns([
    pl.col('Position').cast(pl.Int64, strict=False),
    pl.col('Rank2025').cast(pl.Int64, strict=False),
    pl.col('Year').cast(pl.Int64, strict=False),
    pl.col('Minutes').cast(pl.Int64, strict=False),
    pl.col('Title').str.strip_chars(),
    pl.col('Director').str.strip_chars(),
    pl.col('Country').str.strip_chars()
])

# Remove rows with null values in key columns
df_greatest_films = df_greatest_films.filter(
    pl.col('Position').is_not_null() & 
    pl.col('Title').is_not_null() &
    pl.col('Director').is_not_null()
)

# Drop duplicates
df_greatest_films = df_greatest_films.unique()

print("=" * 60)
print("GREATEST FILMS TABLE (Cleaned)")
print("=" * 60)
print(f"Shape: {df_greatest_films.shape}")
print(f"Columns: {df_greatest_films.columns}")
print("\nData types:")
print(df_greatest_films.dtypes)
print("\nNull counts:")
print(df_greatest_films.null_count())
df_greatest_films.head(10)

GREATEST FILMS TABLE (Cleaned)
Shape: (1000, 7)
Columns: ['Position', 'Rank2025', 'Title', 'Director', 'Year', 'Country', 'Minutes']

Data types:
[Int64, Int64, String, String, Int64, String, Int64]

Null counts:
shape: (1, 7)
┌──────────┬──────────┬───────┬──────────┬──────┬─────────┬─────────┐
│ Position ┆ Rank2025 ┆ Title ┆ Director ┆ Year ┆ Country ┆ Minutes │
│ ---      ┆ ---      ┆ ---   ┆ ---      ┆ ---  ┆ ---     ┆ ---     │
│ u32      ┆ u32      ┆ u32   ┆ u32      ┆ u32  ┆ u32     ┆ u32     │
╞══════════╪══════════╪═══════╪══════════╪══════╪═════════╪═════════╡
│ 0        ┆ 0        ┆ 0     ┆ 0        ┆ 1    ┆ 0       ┆ 0       │
└──────────┴──────────┴───────┴──────────┴──────┴─────────┴─────────┘


Position,Rank2025,Title,Director,Year,Country,Minutes
i64,i64,str,str,i64,str,i64
99,98,"""To Be or Not to Be""","""Lubitsch, Ernst""",1942,"""USA""",99
53,55,"""M""","""Lang, Fritz""",1931,"""Germany""",99
545,527,"""Vampires, Les""","""Feuillade, Louis""",1915,"""France""",421
693,674,"""Bambi""","""Hand, David""",1942,"""USA""",69
118,118,"""Yi Yi""","""Yang, Edward""",2000,"""Taiwan""",173
795,781,"""Foolish Wives""","""von Stroheim, Erich""",1922,"""USA""",117
41,43,"""Sunset Blvd.""","""Wilder, Billy""",1950,"""USA""",110
100,100,"""Meshes of the Afternoon""","""Deren, Maya & Alexander Hammid""",1943,"""USA""",15
356,345,"""Weekend""","""Godard, Jean-Luc""",1967,"""France""",105


---
## Task 3: Create Directors and Countries Dimension Tables
Creating dimension tables for Directors and Countries from the Greatest Films data to connect with Films table via DirectorID and CountryID.

In [7]:
# Create DIRECTORS dimension table
# Extract unique directors from Greatest Films
df_directors = (
    df_greatest_films
    .select('Director')
    .unique()
    .filter(pl.col('Director').is_not_null())
    .sort('Director')
    .with_row_index('DirectorID', offset=1)  # Create DirectorID starting from 1
    .select(['DirectorID', 'Director'])
)

# Convert DirectorID to Int64
df_directors = df_directors.with_columns(
    pl.col('DirectorID').cast(pl.Int64)
)

print("=" * 60)
print("DIRECTORS DIMENSION TABLE")
print("=" * 60)
print(f"Shape: {df_directors.shape}")
print(f"Unique directors: {df_directors.height}")
df_directors.head(15)

DIRECTORS DIMENSION TABLE
Shape: (429, 2)
Unique directors: 429


DirectorID,Director
i64,str
1,"""Abrahams, Jim/David Zucker/Jer…"
2,"""Ade, Maren"""
3,"""Akerman, Chantal"""
4,"""Aldrich, Robert"""
5,"""Alfredson, Tomas"""
…,…
11,"""Anderson, Wes"""
12,"""Andersson, Roy"""
13,"""Angelopoulos, Theo"""


In [8]:
# Create COUNTRIES dimension table
# Extract unique countries from Greatest Films
# Some films have multiple countries separated by '/'
countries_list = (
    df_greatest_films
    .select('Country')
    .filter(pl.col('Country').is_not_null())
    .with_columns(
        pl.col('Country').str.split('/')
    )
    .explode('Country')
    .with_columns(
        pl.col('Country').str.strip_chars()
    )
    .unique()
    .sort('Country')
)

df_countries = (
    countries_list
    .with_row_index('CountryID', offset=1)
    .select(['CountryID', 'Country'])
)

# Convert CountryID to Int64
df_countries = df_countries.with_columns(
    pl.col('CountryID').cast(pl.Int64)
)

print("=" * 60)
print("COUNTRIES DIMENSION TABLE")
print("=" * 60)
print(f"Shape: {df_countries.shape}")
print(f"Unique countries: {df_countries.height}")
df_countries

COUNTRIES DIMENSION TABLE
Shape: (43, 2)
Unique countries: 43


CountryID,Country
i64,str
1,"""Angola"""
2,"""Argentina"""
3,"""Australia"""
4,"""Austria"""
5,"""Belgium"""
…,…
39,"""UK"""
40,"""USA"""
41,"""USSR"""


---
## Task 4: Star Schema Data Model Verification
Verifying that our data model follows Star Schema requirements with Films as the fact table and Certificate, Genre, Directors, and Countries as dimension tables.

In [9]:
print("=" * 70)
print("STAR SCHEMA DATA MODEL VERIFICATION")
print("=" * 70)
print()
print("FACT TABLE:")
print("-" * 40)
print(f"  Films: {df_films.shape[0]} records, {df_films.shape[1]} columns")
print(f"  Measures: BudgetDollars, BoxOfficeDollars, RunTimeMinutes, OscarNominations, OscarWins")
print(f"  Foreign Keys: DirectorID, CountryID, GenreID, CertificateID, StudioID, LanguageID")
print()
print("DIMENSION TABLES:")
print("-" * 40)
print(f"  1. Certificate: {df_certificate.shape[0]} records (CertificateID -> Films.CertificateID)")
print(f"  2. Genre: {df_genre.shape[0]} records (GenreID -> Films.GenreID)")
print(f"  3. Directors: {df_directors.shape[0]} records (DirectorID -> Films.DirectorID)")
print(f"  4. Countries: {df_countries.shape[0]} records (CountryID -> Films.CountryID)")
print()
print("STAR SCHEMA DIAGRAM:")
print("-" * 40)
print("""
                    [Certificate]
                          |
                    CertificateID
                          |
    [Directors]----DirectorID----[FILMS]----GenreID----[Genre]
                          |       (Fact)
                    CountryID
                          |
                    [Countries]
""")
print()
print("Star Schema Requirements:")
print("  [OK] Central fact table (Films) with measures and foreign keys")
print("  [OK] Dimension tables connected via single foreign key")
print("  [OK] Denormalized dimension tables for query performance")
print("  [OK] Each dimension has a primary key matching fact table foreign key")

STAR SCHEMA DATA MODEL VERIFICATION

FACT TABLE:
----------------------------------------
  Films: 1000 records, 15 columns
  Measures: BudgetDollars, BoxOfficeDollars, RunTimeMinutes, OscarNominations, OscarWins
  Foreign Keys: DirectorID, CountryID, GenreID, CertificateID, StudioID, LanguageID

DIMENSION TABLES:
----------------------------------------
  1. Certificate: 7 records (CertificateID -> Films.CertificateID)
  2. Genre: 24 records (GenreID -> Films.GenreID)
  3. Directors: 429 records (DirectorID -> Films.DirectorID)
  4. Countries: 43 records (CountryID -> Films.CountryID)

STAR SCHEMA DIAGRAM:
----------------------------------------

                    [Certificate]
                          |
                    CertificateID
                          |
    [Directors]----DirectorID----[FILMS]----GenreID----[Genre]
                          |       (Fact)
                    CountryID
                          |
                    [Countries]


Star Schema Requirement

In [10]:
# Create enriched Films table by joining all dimensions
df_films_enriched = (
    df_films
    .join(df_genre, on='GenreID', how='left')
    .join(df_certificate, on='CertificateID', how='left')
    .join(df_countries, on='CountryID', how='left')
    .join(df_directors, on='DirectorID', how='left')
)

print("=" * 60)
print("ENRICHED FILMS TABLE (After joining dimensions)")
print("=" * 60)
print(f"Shape: {df_films_enriched.shape}")
print(f"Columns: {df_films_enriched.columns}")
df_films_enriched.head(5)

ENRICHED FILMS TABLE (After joining dimensions)
Shape: (1000, 19)
Columns: ['FilmID', 'Title', 'ReleaseDate', 'DirectorID', 'StudioID', 'Review', 'CountryID', 'LanguageID', 'GenreID', 'RunTimeMinutes', 'CertificateID', 'BudgetDollars', 'BoxOfficeDollars', 'OscarNominations', 'OscarWins', 'Genre', 'Certificate', 'Country', 'Director']


FilmID,Title,ReleaseDate,DirectorID,StudioID,Review,CountryID,LanguageID,GenreID,RunTimeMinutes,CertificateID,BudgetDollars,BoxOfficeDollars,OscarNominations,OscarWins,Genre,Certificate,Country,Director
i64,str,date,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str
1,"""Jurassic Park""",1993-06-11,4,41,"""People clone dinosaurs. Dinosa…",241,1,8,126,2,63000000,1029939903,3,3,"""Adventure""","""PG""",null,"""Aldrich, Robert"""
2,"""Spider-Man""",2002-05-03,11,53,"""High-school nerd Peter Parker …",241,1,4,121,4,140000000,821708551,2,0,"""Action""","""12A""",null,"""Anderson, Wes"""
3,"""King Kong""",2005-12-14,12,42,"""Dizzy blonde actress falls for…",167,1,8,187,4,207000000,550500000,4,3,"""Adventure""","""12A""",null,"""Andersson, Roy"""
5,"""Superman Returns""",2006-07-14,14,43,"""The Man of Steel returns from …",241,1,4,154,4,204000000,391081192,1,0,"""Action""","""12A""",null,"""Anger, Kenneth"""
6,"""Titanic""",1998-01-23,15,4,"""Bleurgh! Sorry, that's the sou…",241,1,6,194,3,200000000,2186772302,14,11,"""Romance""","""12""",null,"""Antonioni, Michelangelo"""


---
## Task 5: Genre Analysis - Longest Film Duration
Finding the genre(s) with the longest average film duration and visualizing the results.

In [11]:
# Calculate average runtime by genre
genre_duration = (
    df_films_enriched
    .group_by('Genre')
    .agg([
        pl.col('RunTimeMinutes').mean().alias('AvgDuration'),
        pl.col('RunTimeMinutes').max().alias('MaxDuration'),
        pl.col('RunTimeMinutes').min().alias('MinDuration'),
        pl.len().alias('FilmCount')
    ])
    .sort('AvgDuration', descending=True)
    .with_columns([
        pl.col('AvgDuration').round(1)
    ])
)

# Find the genre(s) with longest duration
max_duration = genre_duration['AvgDuration'].max()
longest_genres = genre_duration.filter(pl.col('AvgDuration') == max_duration)

print("=" * 60)
print("GENRE WITH LONGEST AVERAGE FILM DURATION")
print("=" * 60)
print(f"\nLongest average duration: {max_duration} minutes")
print(f"Genre(s): {longest_genres['Genre'].to_list()}")
print("\nAll genres by average duration:")
genre_duration

GENRE WITH LONGEST AVERAGE FILM DURATION

Longest average duration: 166.7 minutes
Genre(s): ['History']

All genres by average duration:


Genre,AvgDuration,MaxDuration,MinDuration,FilmCount
str,f64,i64,i64,u32
"""History""",166.7,248,105,11
"""Biography""",147.3,222,93,17
"""War""",144.0,183,88,35
"""Fantasy""",137.9,220,89,35
"""Musical""",132.9,179,87,21
…,…,…,…,…
"""Comedy""",106.2,210,68,113
"""Family""",105.2,120,89,5
"""Martial Arts""",105.0,153,78,67


In [12]:
# Visualization: Genre Duration Analysis
fig_genre = go.Figure()

# Create color list - highlight the longest genre
colors = [PRODIVI_COLORS['accent'] if d == max_duration else PRODIVI_COLORS['secondary'] 
          for d in genre_duration['AvgDuration'].to_list()]

fig_genre.add_trace(go.Bar(
    x=genre_duration['Genre'].to_list(),
    y=genre_duration['AvgDuration'].to_list(),
    marker_color=colors,
    text=genre_duration['AvgDuration'].to_list(),
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>' +
                  'Avg Duration: %{y:.1f} min<br>' +
                  '<extra></extra>'
))

fig_genre.update_layout(
    title={
        'text': 'Average Film Duration by Genre',
        'font': {'size': 20, 'color': PRODIVI_COLORS['text']}
    },
    xaxis_title='Genre',
    yaxis_title='Average Duration (minutes)',
    xaxis_tickangle=-45,
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    showlegend=False,
    height=500
)

# Add annotation for longest genre
fig_genre.add_annotation(
    x=longest_genres['Genre'][0],
    y=max_duration + 5,
    text=f"Longest: {max_duration} min",
    showarrow=True,
    arrowhead=2,
    arrowcolor=PRODIVI_COLORS['accent'],
    font={'color': PRODIVI_COLORS['accent'], 'size': 12}
)

fig_genre.show()

---
## Task 6: Budget Analysis by Genre
Calculating average film budget per genre, finding percentage of films exceeding average budget, and highlighting genres exceeding 30%.

**Note:** The Countries dimension table created from Greatest Films has different IDs than the Films table, so we analyze by Genre instead (which has matching IDs).

In [13]:
# Calculate average budget per genre
genre_budget = (
    df_films_enriched
    .filter(pl.col('BudgetDollars').is_not_null() & (pl.col('BudgetDollars') > 0))
    .group_by('Genre')
    .agg([
        pl.col('BudgetDollars').mean().alias('AvgBudget'),
        pl.col('BudgetDollars').count().alias('TotalFilms')
    ])
    .filter(pl.col('Genre').is_not_null())
)

# Calculate percentage of films exceeding average budget per genre
df_with_genre_avg = (
    df_films_enriched
    .filter(pl.col('BudgetDollars').is_not_null() & (pl.col('BudgetDollars') > 0))
    .join(genre_budget.select(['Genre', 'AvgBudget']), on='Genre', how='left')
)

# Calculate films exceeding average
budget_analysis = (
    df_with_genre_avg
    .group_by('Genre')
    .agg([
        pl.col('BudgetDollars').mean().alias('AvgBudget'),
        pl.len().alias('TotalFilms'),
        (pl.col('BudgetDollars') > pl.col('AvgBudget')).sum().alias('FilmsAboveAvg')
    ])
    .filter(pl.col('Genre').is_not_null())
    .with_columns([
        (pl.col('FilmsAboveAvg') / pl.col('TotalFilms') * 100).round(1).alias('PctAboveAvg'),
        (pl.col('AvgBudget') / 1_000_000).round(2).alias('AvgBudgetM')  # In millions
    ])
    .with_columns([
        pl.when(pl.col('PctAboveAvg') > 30)
        .then(pl.lit('Above 30%'))
        .otherwise(pl.lit('Below 30%'))
        .alias('Category')
    ])
    .sort('PctAboveAvg', descending=True)
)

print("=" * 70)
print("BUDGET ANALYSIS BY GENRE")
print("=" * 70)
print("\nGenres with percentage of films exceeding average budget:")
budget_analysis

BUDGET ANALYSIS BY GENRE

Genres with percentage of films exceeding average budget:


Genre,AvgBudget,TotalFilms,FilmsAboveAvg,PctAboveAvg,AvgBudgetM,Category
str,f64,u32,u32,f64,f64,str
"""Family""",6.36e7,5,3,60.0,63.6,"""Above 30%"""
"""Biography""",4.8474e7,17,9,52.9,48.47,"""Above 30%"""
"""Awful""",8.7e7,4,2,50.0,87.0,"""Above 30%"""
"""Documentary""",532500.0,2,1,50.0,0.53,"""Above 30%"""
"""Animation""",9.9292e7,63,31,49.2,99.29,"""Above 30%"""
…,…,…,…,…,…,…
"""Horror""",3.1097e7,21,6,28.6,31.1,"""Below 30%"""
"""Drama""",1.8095e7,73,20,27.4,18.09,"""Below 30%"""
"""Western""",9.5851e6,23,6,26.1,9.59,"""Below 30%"""


In [14]:
# Visualization: Budget Analysis by Genre
# Color based on whether percentage exceeds 30%
colors_budget = [PRODIVI_COLORS['accent'] if pct > 30 else PRODIVI_COLORS['secondary'] 
                 for pct in budget_analysis['PctAboveAvg'].to_list()]

fig_budget = go.Figure()

fig_budget.add_trace(go.Bar(
    x=budget_analysis['Genre'].to_list(),
    y=budget_analysis['PctAboveAvg'].to_list(),
    marker_color=colors_budget,
    text=[f"{p}%" for p in budget_analysis['PctAboveAvg'].to_list()],
    textposition='outside',
    customdata=budget_analysis[['AvgBudget', 'TotalFilms', 'FilmsAboveAvg']].to_numpy(),
    hovertemplate='<b>%{x}</b><br>' +
                  'Films Above Avg: %{y:.1f}%<br>' +
                  'Average Budget: $%{customdata[0]:,.0f}<br>' +
                  'Total Films: %{customdata[1]}<br>' +
                  'Films Above Avg: %{customdata[2]}<br>' +
                  '<extra></extra>'
))

# Add 30% threshold line
fig_budget.add_hline(
    y=30, 
    line_dash="dash", 
    line_color=PRODIVI_COLORS['warning'],
    annotation_text="30% Threshold",
    annotation_position="right"
)

fig_budget.update_layout(
    title={
        'text': 'Percentage of Films Exceeding Average Budget by Genre',
        'font': {'size': 18, 'color': PRODIVI_COLORS['text']}
    },
    xaxis_title='Genre',
    yaxis_title='Percentage Above Average Budget (%)',
    xaxis_tickangle=-45,
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    showlegend=False,
    height=500,
    annotations=[
        dict(
            x=0.02, y=0.98,
            xref='paper', yref='paper',
            text='<b style="color:' + PRODIVI_COLORS['accent'] + '">Red</b> = Above 30% threshold',
            showarrow=False,
            font={'size': 11},
            bgcolor='white',
            bordercolor=PRODIVI_COLORS['neutral'],
            borderwidth=1
        )
    ]
)

fig_budget.show()

In [15]:
# Additional view: Average Budget by Genre (actual numbers)
fig_avg_budget = px.treemap(
    budget_analysis.to_pandas(),
    path=['Genre'],
    values='TotalFilms',
    color='AvgBudgetM',
    color_continuous_scale=[[0, PRODIVI_COLORS['secondary']], [1, PRODIVI_COLORS['accent']]],
    hover_data={'AvgBudget': ':$,.0f', 'PctAboveAvg': ':.1f%'},
    title='Average Film Budget by Genre (Hover to see actual budget)'
)

fig_avg_budget.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Films: %{value}<br>' +
                  'Avg Budget: $%{customdata[0]:,.0f}<br>' +
                  'Above Avg: %{customdata[1]:.1f}%<extra></extra>'
)

fig_avg_budget.update_layout(
    font={'color': PRODIVI_COLORS['text']},
    paper_bgcolor='white',
    height=450
)

fig_avg_budget.show()

---
## Task 7: Monthly Box Office Earnings by Year
Calculating average monthly earnings per year from BoxOfficeDollars.

In [16]:
# Calculate average monthly earnings per year
# Extract year from ReleaseDate
yearly_earnings = (
    df_films_enriched
    .filter(pl.col('BoxOfficeDollars').is_not_null() & (pl.col('BoxOfficeDollars') > 0))
    .with_columns([
        pl.col('ReleaseDate').dt.year().alias('Year'),
        pl.col('ReleaseDate').dt.month().alias('Month')
    ])
    .group_by('Year')
    .agg([
        pl.col('BoxOfficeDollars').sum().alias('TotalBoxOffice'),
        pl.col('BoxOfficeDollars').count().alias('FilmCount'),
        pl.col('Month').n_unique().alias('ActiveMonths')
    ])
    .filter(pl.col('Year').is_not_null())
    .with_columns([
        # Average monthly earnings = Total / 12 months
        (pl.col('TotalBoxOffice') / 12).round(0).alias('AvgMonthlyEarnings'),
        (pl.col('TotalBoxOffice') / 1_000_000).round(2).alias('TotalBoxOfficeM')
    ])
    .sort('Year')
)

print("=" * 70)
print("AVERAGE MONTHLY BOX OFFICE EARNINGS BY YEAR")
print("=" * 70)
yearly_earnings

AVERAGE MONTHLY BOX OFFICE EARNINGS BY YEAR


Year,TotalBoxOffice,FilmCount,ActiveMonths,AvgMonthlyEarnings,TotalBoxOfficeM
i32,i64,u32,u32,f64,f64
1921,2500000,1,1,208333.0,2.5
1930,3000000,1,1,250000.0,3.0
1931,12000000,1,1,1e6,12.0
1932,600000,1,1,50000.0,0.6
1933,2847000,1,1,237250.0,2.85
…,…,…,…,…,…
2012,11691612166,21,12,9.74301014e8,11691.61
2013,10616600000,21,11,8.84716667e8,10616.6
2014,9543000000,18,10,7.9525e8,9543.0


In [17]:
# Visualization: Average Monthly Earnings by Year
fig_earnings = go.Figure()

fig_earnings.add_trace(go.Scatter(
    x=yearly_earnings['Year'].to_list(),
    y=yearly_earnings['AvgMonthlyEarnings'].to_list(),
    mode='lines+markers',
    line={'color': PRODIVI_COLORS['primary'], 'width': 3},
    marker={'size': 10, 'color': PRODIVI_COLORS['secondary']},
    fill='tozeroy',
    fillcolor='rgba(52, 152, 219, 0.2)',
    customdata=yearly_earnings[['TotalBoxOffice', 'FilmCount']].to_numpy(),
    hovertemplate='<b>Year %{x}</b><br>' +
                  'Avg Monthly: $%{y:,.0f}<br>' +
                  'Total Annual: $%{customdata[0]:,.0f}<br>' +
                  'Films: %{customdata[1]}<br>' +
                  '<extra></extra>'
))

fig_earnings.update_layout(
    title={
        'text': 'Average Monthly Box Office Earnings by Year',
        'font': {'size': 20, 'color': PRODIVI_COLORS['text']}
    },
    xaxis_title='Year',
    yaxis_title='Average Monthly Earnings ($)',
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    hovermode='x unified',
    height=450,
    yaxis_tickformat='$,.0f'
)

fig_earnings.show()

In [18]:
# Bar chart version for clearer comparison
fig_earnings_bar = go.Figure()

fig_earnings_bar.add_trace(go.Bar(
    x=yearly_earnings['Year'].to_list(),
    y=[e/1_000_000 for e in yearly_earnings['AvgMonthlyEarnings'].to_list()],
    marker_color=PRODIVI_COLORS['secondary'],
    marker_line_color=PRODIVI_COLORS['primary'],
    marker_line_width=1,
    customdata=yearly_earnings[['TotalBoxOffice', 'FilmCount', 'AvgMonthlyEarnings']].to_numpy(),
    hovertemplate='<b>Year %{x}</b><br>' +
                  'Avg Monthly: $%{customdata[2]:,.0f}<br>' +
                  'Total Annual: $%{customdata[0]:,.0f}<br>' +
                  'Films: %{customdata[1]}<br>' +
                  '<extra></extra>'
))

fig_earnings_bar.update_layout(
    title={
        'text': 'Average Monthly Box Office Earnings by Year',
        'font': {'size': 20, 'color': PRODIVI_COLORS['text']}
    },
    xaxis_title='Year',
    yaxis_title='Average Monthly Earnings ($ Millions)',
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    height=450
)

fig_earnings_bar.show()

---
## Task 8: Custom Meaningful Visualization
Creating additional visualizations to provide deeper insights into the film data.

In [19]:
# Oscar Analysis: Success Rate by Genre
oscar_analysis = (
    df_films_enriched
    .group_by('Genre')
    .agg([
        pl.len().alias('TotalFilms'),
        pl.col('OscarNominations').sum().alias('TotalNominations'),
        pl.col('OscarWins').sum().alias('TotalWins'),
        pl.col('OscarNominations').mean().round(2).alias('AvgNominations'),
        pl.col('OscarWins').mean().round(2).alias('AvgWins'),
        pl.col('BudgetDollars').mean().alias('AvgBudget'),
        pl.col('BoxOfficeDollars').mean().alias('AvgBoxOffice')
    ])
    .filter(pl.col('Genre').is_not_null())
    .with_columns([
        pl.when(pl.col('TotalNominations') > 0)
        .then((pl.col('TotalWins') / pl.col('TotalNominations') * 100).round(1))
        .otherwise(0)
        .alias('WinRate')
    ])
    .sort('TotalWins', descending=True)
)

print("=" * 70)
print("OSCAR ANALYSIS BY GENRE")
print("=" * 70)
oscar_analysis

OSCAR ANALYSIS BY GENRE


Genre,TotalFilms,TotalNominations,TotalWins,AvgNominations,AvgWins,AvgBudget,AvgBoxOffice,WinRate
str,u32,i64,i64,f64,f64,f64,f64,f64
"""Drama""",82,334,107,4.07,1.3,1.8095e7,1.1673e8,32.0
"""Musical""",21,109,53,5.19,2.52,1.070005e7,9.8693e7,48.6
"""Science Fiction""",92,176,51,1.91,0.55,8.6163e7,3.8316e8,29.0
"""Adventure""",73,136,41,1.86,0.56,7.9250e7,3.3764e8,30.1
"""Crime""",51,122,40,2.39,0.78,1.9417e7,7.4450e7,32.8
…,…,…,…,…,…,…,…,…
"""Martial Arts""",67,12,4,0.18,0.06,2.2215e7,1.2132e8,33.3
"""Documentary""",2,1,0,0.5,0.0,532500.0,2.9529368e7,0.0
"""Awful""",4,0,0,0.0,0.0,8.7e7,7.37525e8,0.0


In [20]:
# Visualization: Oscar Success by Genre
fig_oscar = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Total Oscar Nominations & Wins', 'Win Rate (%)'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Left: Nominations and Wins
fig_oscar.add_trace(
    go.Bar(
        name='Nominations',
        x=oscar_analysis['Genre'].to_list(),
        y=oscar_analysis['TotalNominations'].to_list(),
        marker_color=PRODIVI_COLORS['secondary']
    ),
    row=1, col=1
)

fig_oscar.add_trace(
    go.Bar(
        name='Wins',
        x=oscar_analysis['Genre'].to_list(),
        y=oscar_analysis['TotalWins'].to_list(),
        marker_color=PRODIVI_COLORS['warning']
    ),
    row=1, col=1
)

# Right: Win Rate
fig_oscar.add_trace(
    go.Bar(
        name='Win Rate',
        x=oscar_analysis['Genre'].to_list(),
        y=oscar_analysis['WinRate'].to_list(),
        marker_color=PRODIVI_COLORS['success'],
        showlegend=False
    ),
    row=1, col=2
)

fig_oscar.update_layout(
    title={
        'text': 'Oscar Performance by Genre',
        'font': {'size': 20, 'color': PRODIVI_COLORS['text']}
    },
    barmode='group',
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    height=500,
    legend={'orientation': 'h', 'y': -0.2}
)

fig_oscar.update_xaxes(tickangle=-45)
fig_oscar.show()

In [21]:
# ROI Analysis: Budget vs Box Office by Genre
roi_analysis = (
    df_films_enriched
    .filter(
        pl.col('BudgetDollars').is_not_null() & 
        pl.col('BoxOfficeDollars').is_not_null() &
        (pl.col('BudgetDollars') > 0)
    )
    .with_columns([
        ((pl.col('BoxOfficeDollars') - pl.col('BudgetDollars')) / pl.col('BudgetDollars') * 100)
        .alias('ROI')
    ])
    .group_by('Genre')
    .agg([
        pl.col('ROI').mean().round(1).alias('AvgROI'),
        pl.col('BudgetDollars').mean().alias('AvgBudget'),
        pl.col('BoxOfficeDollars').mean().alias('AvgBoxOffice'),
        pl.len().alias('FilmCount')
    ])
    .filter(pl.col('Genre').is_not_null())
    .sort('AvgROI', descending=True)
)

print("=" * 70)
print("ROI ANALYSIS BY GENRE")
print("=" * 70)
roi_analysis

ROI ANALYSIS BY GENRE


Genre,AvgROI,AvgBudget,AvgBoxOffice,FilmCount
str,f64,f64,f64,u32
"""Documentary""",45329.8,65000.0,2.9529368e7,1
"""Horror""",2590.0,3.2634e7,1.4108e8,20
"""Romantic Comedy""",2371.4,1.536875e7,1.7041e8,8
"""Romance""",2310.4,3.06e7,3.5865e8,14
"""Western""",1703.3,9.5851e6,5.7835e7,23
…,…,…,…,…
"""Mystery""",389.9,9.6714e7,4.8514e8,7
"""War""",360.1,3.1872e7,8.9466496e7,32
"""Action""",345.8,9.5220e7,3.5149e8,114


In [22]:
# ROI Visualization
fig_roi = go.Figure()

# Color by ROI (positive = green, negative = red)
colors_roi = [PRODIVI_COLORS['success'] if roi > 0 else PRODIVI_COLORS['accent'] 
              for roi in roi_analysis['AvgROI'].to_list()]

fig_roi.add_trace(go.Bar(
    x=roi_analysis['Genre'].to_list(),
    y=roi_analysis['AvgROI'].to_list(),
    marker_color=colors_roi,
    text=[f"{r:.0f}%" for r in roi_analysis['AvgROI'].to_list()],
    textposition='outside',
    customdata=roi_analysis[['AvgBudget', 'AvgBoxOffice', 'FilmCount']].to_numpy(),
    hovertemplate='<b>%{x}</b><br>' +
                  'ROI: %{y:.1f}%<br>' +
                  'Avg Budget: $%{customdata[0]:,.0f}<br>' +
                  'Avg Box Office: $%{customdata[1]:,.0f}<br>' +
                  'Films: %{customdata[2]}<br>' +
                  '<extra></extra>'
))

fig_roi.add_hline(y=0, line_color=PRODIVI_COLORS['neutral'], line_width=2)

fig_roi.update_layout(
    title={
        'text': 'Average Return on Investment (ROI) by Genre',
        'font': {'size': 20, 'color': PRODIVI_COLORS['text']}
    },
    xaxis_title='Genre',
    yaxis_title='Average ROI (%)',
    xaxis_tickangle=-45,
    plot_bgcolor=PRODIVI_COLORS['background'],
    paper_bgcolor='white',
    font={'color': PRODIVI_COLORS['text']},
    height=500
)

fig_roi.show()

---
## Task 9: Interactive Report Dashboard with Filters
Creating an interactive dashboard with slicers/filters for the presentation.

In [23]:
# Key Performance Indicators Summary
print("=" * 70)
print("FILMS ANALYTICS DASHBOARD - KEY METRICS")
print("=" * 70)

total_films = df_films_enriched.height
total_budget = df_films_enriched['BudgetDollars'].sum()
total_box_office = df_films_enriched['BoxOfficeDollars'].sum()
total_oscars = df_films_enriched['OscarWins'].sum()
avg_runtime = df_films_enriched['RunTimeMinutes'].mean()

print(f"\nTotal Films: {total_films:,}")
print(f"Total Budget: ${total_budget:,.0f}")
print(f"Total Box Office: ${total_box_office:,.0f}")
print(f"Total Oscar Wins: {total_oscars:,}")
print(f"Average Runtime: {avg_runtime:.0f} minutes")
print(f"Overall ROI: {((total_box_office - total_budget) / total_budget * 100):.1f}%")

FILMS ANALYTICS DASHBOARD - KEY METRICS

Total Films: 1,000
Total Budget: $49,826,897,674
Total Box Office: $231,474,920,799
Total Oscar Wins: 590
Average Runtime: 120 minutes
Overall ROI: 364.6%


In [24]:
# Interactive Dashboard with Multiple Views
from ipywidgets import interact, widgets, interactive_output, VBox, HBox
import IPython.display as display

# Get unique values for filters
genres = ['All'] + sorted(df_films_enriched['Genre'].unique().drop_nulls().to_list())
certificates = ['All'] + sorted(df_films_enriched['Certificate'].unique().drop_nulls().to_list())
countries = ['All'] + sorted(df_films_enriched['Country'].unique().drop_nulls().to_list())

# Get year range
years = df_films_enriched.with_columns(
    pl.col('ReleaseDate').dt.year().alias('Year')
)['Year'].drop_nulls()
min_year = int(years.min())
max_year = int(years.max())

print("Available Filters:")
print(f"- Genres: {len(genres)-1} options")
print(f"- Certificates: {len(certificates)-1} options")
print(f"- Countries: {len(countries)-1} options")
print(f"- Year Range: {min_year} - {max_year}")

Available Filters:
- Genres: 24 options
- Certificates: 7 options
- Countries: 0 options
- Year Range: 1921 - 2016


In [25]:
def create_filtered_dashboard(genre_filter='All', certificate_filter='All', 
                             country_filter='All', year_start=None, year_end=None):
    """Create dashboard with applied filters"""
    
    # Apply filters
    df_filtered = df_films_enriched.with_columns(
        pl.col('ReleaseDate').dt.year().alias('Year')
    )
    
    if genre_filter != 'All':
        df_filtered = df_filtered.filter(pl.col('Genre') == genre_filter)
    if certificate_filter != 'All':
        df_filtered = df_filtered.filter(pl.col('Certificate') == certificate_filter)
    if country_filter != 'All':
        df_filtered = df_filtered.filter(pl.col('Country') == country_filter)
    if year_start and year_end:
        df_filtered = df_filtered.filter(
            (pl.col('Year') >= year_start) & (pl.col('Year') <= year_end)
        )
    
    # Calculate KPIs
    n_films = df_filtered.height
    avg_budget = df_filtered['BudgetDollars'].mean() or 0
    avg_box_office = df_filtered['BoxOfficeDollars'].mean() or 0
    total_oscars = df_filtered['OscarWins'].sum() or 0
    avg_runtime = df_filtered['RunTimeMinutes'].mean() or 0
    
    # Create KPI cards figure
    fig_kpi = go.Figure()
    
    kpi_values = [
        (f"{n_films:,}", "Total Films", PRODIVI_COLORS['primary']),
        (f"${avg_budget/1e6:.1f}M", "Avg Budget", PRODIVI_COLORS['secondary']),
        (f"${avg_box_office/1e6:.1f}M", "Avg Box Office", PRODIVI_COLORS['success']),
        (f"{total_oscars:,}", "Oscar Wins", PRODIVI_COLORS['warning']),
        (f"{avg_runtime:.0f} min", "Avg Runtime", PRODIVI_COLORS['accent'])
    ]
    
    for i, (value, label, color) in enumerate(kpi_values):
        fig_kpi.add_trace(go.Indicator(
            mode="number",
            value=float(value.replace('$', '').replace('M', '').replace(',', '').replace(' min', '')) if value[0].isdigit() or value[0] == '$' else 0,
            title={'text': f"<b>{label}</b>", 'font': {'size': 14}},
            number={'font': {'size': 24, 'color': color}, 'suffix': 'M' if 'M' in value else (' min' if 'min' in value else '')},
            domain={'x': [i/5, (i+1)/5], 'y': [0, 1]}
        ))
    
    fig_kpi.update_layout(
        height=120,
        paper_bgcolor=PRODIVI_COLORS['background'],
        margin=dict(l=20, r=20, t=30, b=20)
    )
    
    return df_filtered, fig_kpi

# Test with no filters
df_test, fig_kpi = create_filtered_dashboard()
print(f"Filtered dataset: {df_test.height} films")

Filtered dataset: 1000 films


In [26]:
# Full Interactive Dashboard
def show_dashboard(genre='All', certificate='All', country='All', year_range=(min_year, max_year)):
    """Display full interactive dashboard"""
    
    # Filter data
    df_filtered = df_films_enriched.with_columns(
        pl.col('ReleaseDate').dt.year().alias('Year')
    )
    
    if genre != 'All':
        df_filtered = df_filtered.filter(pl.col('Genre') == genre)
    if certificate != 'All':
        df_filtered = df_filtered.filter(pl.col('Certificate') == certificate)
    if country != 'All':
        df_filtered = df_filtered.filter(pl.col('Country') == country)
    
    df_filtered = df_filtered.filter(
        (pl.col('Year') >= year_range[0]) & (pl.col('Year') <= year_range[1])
    )
    
    if df_filtered.height == 0:
        print("No data matches the selected filters.")
        return
    
    # KPIs
    n_films = df_filtered.height
    avg_budget = df_filtered['BudgetDollars'].mean() or 0
    avg_box_office = df_filtered['BoxOfficeDollars'].mean() or 0
    total_oscars = df_filtered['OscarWins'].sum() or 0
    
    print("=" * 70)
    print("FILTERED RESULTS")
    print("=" * 70)
    print(f"Films: {n_films:,} | Avg Budget: ${avg_budget/1e6:.1f}M | Avg Box Office: ${avg_box_office/1e6:.1f}M | Oscars: {total_oscars}")
    print()
    
    # Create subplot dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Films by Year', 'Budget vs Box Office',
            'Genre Distribution', 'Oscar Performance'
        ),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "pie"}, {"type": "bar"}]]
    )
    
    # 1. Films by Year
    yearly = df_filtered.group_by('Year').len().sort('Year')
    fig.add_trace(
        go.Bar(x=yearly['Year'].to_list(), y=yearly['len'].to_list(),
               marker_color=PRODIVI_COLORS['secondary'], name='Films'),
        row=1, col=1
    )
    
    # 2. Budget vs Box Office scatter
    scatter_data = df_filtered.filter(
        pl.col('BudgetDollars').is_not_null() & pl.col('BoxOfficeDollars').is_not_null()
    )
    fig.add_trace(
        go.Scatter(
            x=[b/1e6 for b in scatter_data['BudgetDollars'].to_list()],
            y=[b/1e6 for b in scatter_data['BoxOfficeDollars'].to_list()],
            mode='markers',
            marker={'color': PRODIVI_COLORS['accent'], 'size': 6, 'opacity': 0.6},
            name='Films',
            text=scatter_data['Title'].to_list(),
            hovertemplate='<b>%{text}</b><br>Budget: $%{x:.1f}M<br>Box Office: $%{y:.1f}M<extra></extra>'
        ),
        row=1, col=2
    )
    
    # 3. Genre distribution
    genre_dist = df_filtered.group_by('Genre').len().filter(pl.col('Genre').is_not_null())
    fig.add_trace(
        go.Pie(
            labels=genre_dist['Genre'].to_list(),
            values=genre_dist['len'].to_list(),
            hole=0.4,
            marker={'colors': px.colors.qualitative.Set2}
        ),
        row=2, col=1
    )
    
    # 4. Oscar performance by certificate
    oscar_cert = df_filtered.group_by('Certificate').agg([
        pl.col('OscarNominations').sum().alias('Nominations'),
        pl.col('OscarWins').sum().alias('Wins')
    ]).filter(pl.col('Certificate').is_not_null()).sort('Wins', descending=True)
    
    fig.add_trace(
        go.Bar(x=oscar_cert['Certificate'].to_list(), y=oscar_cert['Nominations'].to_list(),
               name='Nominations', marker_color=PRODIVI_COLORS['secondary']),
        row=2, col=2
    )
    fig.add_trace(
        go.Bar(x=oscar_cert['Certificate'].to_list(), y=oscar_cert['Wins'].to_list(),
               name='Wins', marker_color=PRODIVI_COLORS['warning']),
        row=2, col=2
    )
    
    fig.update_layout(
        height=700,
        showlegend=True,
        title_text=f'Films Analytics Dashboard (Filtered: {n_films} films)',
        title_font_size=20,
        plot_bgcolor=PRODIVI_COLORS['background'],
        paper_bgcolor='white',
        barmode='group'
    )
    
    fig.update_xaxes(title_text='Year', row=1, col=1)
    fig.update_xaxes(title_text='Budget ($M)', row=1, col=2)
    fig.update_yaxes(title_text='Films', row=1, col=1)
    fig.update_yaxes(title_text='Box Office ($M)', row=1, col=2)
    
    fig.show()

# Create interactive widgets
genre_dropdown = widgets.Dropdown(options=genres, value='All', description='Genre:')
cert_dropdown = widgets.Dropdown(options=certificates, value='All', description='Certificate:')
country_dropdown = widgets.Dropdown(options=countries, value='All', description='Country:')
year_slider = widgets.IntRangeSlider(
    value=[min_year, max_year],
    min=min_year,
    max=max_year,
    step=1,
    description='Years:',
    continuous_update=False
)

# Display interactive dashboard
interact(
    show_dashboard,
    genre=genre_dropdown,
    certificate=cert_dropdown,
    country=country_dropdown,
    year_range=year_slider
);

interactive(children=(Dropdown(description='Genre:', options=('All', 'Action', 'Adventure', 'Animation', 'Awfu…

---
## Summary Statistics and Data Export

In [28]:
# Final Summary
print("=" * 70)
print("FILMS ANALYSIS SUMMARY REPORT")
print("=" * 70)
print("\nDATA MODEL (Star Schema):")
print(f"  - Fact Table: Films ({df_films.height} records)")
print(f"  - Dimension: Certificate ({df_certificate.height} records)")
print(f"  - Dimension: Genre ({df_genre.height} records)")
print(f"  - Dimension: Directors ({df_directors.height} records)")
print(f"  - Dimension: Countries ({df_countries.height} records)")
print(f"\nGREATEST FILMS REFERENCE: {df_greatest_films.height} films")

print("\n" + "=" * 70)
print("KEY FINDINGS:")
print("=" * 70)

# Longest genre
print(f"\n1. LONGEST GENRE: {longest_genres['Genre'][0]} ({max_duration} min avg)")

# Genres above 30% threshold
above_30 = budget_analysis.filter(pl.col('PctAboveAvg') > 30)
print(f"\n2. GENRES EXCEEDING 30% ABOVE-AVERAGE BUDGET THRESHOLD:")
for row in above_30.iter_rows(named=True):
    print(f"   - {row['Genre']}: {row['PctAboveAvg']}% (Avg Budget: ${row['AvgBudget']:,.0f})")

# Best year for monthly earnings
best_year = yearly_earnings.sort('AvgMonthlyEarnings', descending=True).head(1)
print(f"\n3. BEST YEAR FOR MONTHLY EARNINGS: {best_year['Year'][0]}")
print(f"   Average Monthly: ${best_year['AvgMonthlyEarnings'][0]:,.0f}")

# Best ROI genre
best_roi = roi_analysis.head(1)
print(f"\n4. BEST ROI GENRE: {best_roi['Genre'][0]} ({best_roi['AvgROI'][0]}% ROI)")

# Most Oscar wins
most_oscars = oscar_analysis.head(1)
print(f"\n5. MOST OSCAR WINS: {most_oscars['Genre'][0]} ({most_oscars['TotalWins'][0]} wins)")

FILMS ANALYSIS SUMMARY REPORT

DATA MODEL (Star Schema):
  - Fact Table: Films (1000 records)
  - Dimension: Certificate (7 records)
  - Dimension: Genre (24 records)
  - Dimension: Directors (429 records)
  - Dimension: Countries (43 records)

GREATEST FILMS REFERENCE: 1000 films

KEY FINDINGS:

1. LONGEST GENRE: History (166.7 min avg)

2. GENRES EXCEEDING 30% ABOVE-AVERAGE BUDGET THRESHOLD:
   - Family: 60.0% (Avg Budget: $63,600,000)
   - Biography: 52.9% (Avg Budget: $48,473,529)
   - Awful: 50.0% (Avg Budget: $87,000,000)
   - Documentary: 50.0% (Avg Budget: $532,500)
   - Animation: 49.2% (Avg Budget: $99,292,063)
   - Fantasy: 47.1% (Avg Budget: $111,050,000)
   - Mystery: 42.9% (Avg Budget: $96,714,286)
   - Sport: 42.9% (Avg Budget: $23,928,571)
   - Adventure: 42.2% (Avg Budget: $79,250,328)
   - Science Fiction: 41.8% (Avg Budget: $86,162,637)
   - Action: 41.5% (Avg Budget: $92,135,381)
   - Thriller: 38.5% (Avg Budget: $50,620,615)
   - Romantic Comedy: 37.5% (Avg Budget:

In [29]:
# Export dimension tables for reference
print("\nDimension Tables Created:")
print("\n--- DIRECTORS ---")
print(df_directors.head(10))
print(f"... and {df_directors.height - 10} more")

print("\n--- COUNTRIES ---")
print(df_countries)


Dimension Tables Created:

--- DIRECTORS ---
shape: (10, 2)
┌────────────┬─────────────────────────────────┐
│ DirectorID ┆ Director                        │
│ ---        ┆ ---                             │
│ i64        ┆ str                             │
╞════════════╪═════════════════════════════════╡
│ 1          ┆ Abrahams, Jim/David Zucker/Jer… │
│ 2          ┆ Ade, Maren                      │
│ 3          ┆ Akerman, Chantal                │
│ 4          ┆ Aldrich, Robert                 │
│ 5          ┆ Alfredson, Tomas                │
│ 6          ┆ Allen, Woody                    │
│ 7          ┆ Almodóvar, Pedro                │
│ 8          ┆ Altman, Robert                  │
│ 9          ┆ Anderson, Lindsay               │
│ 10         ┆ Anderson, Paul Thomas           │
└────────────┴─────────────────────────────────┘
... and 419 more

--- COUNTRIES ---
shape: (43, 2)
┌───────────┬──────────────┐
│ CountryID ┆ Country      │
│ ---       ┆ ---          │
│ i64       ┆ str